<a href="https://colab.research.google.com/github/rhwans/rhwans/blob/main/bert_%ED%86%A0%ED%94%BD_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리 설치
!pip install contextualized-topic-models==2.2.0 pyldavis pandas

In [2]:
# MeCab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
%cd ..

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 18.0 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2024-10-13 17:37:26--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.142.24, 104.192.142.25, 104.192.142.26, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20filename%3D%22mecab-0.996-ko-0.9.2.tar.gz%22&response-content-encoding=None&AWS

In [3]:
# 필요한 라이브러리 임포트
import pandas as pd
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation, bert_embeddings_from_list
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab
from tqdm import tqdm
import pyLDAvis as vis

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
# CSV 파일 업로드 (Google Colab 사용 시)
from google.colab import files
uploaded = files.upload()

Saving TP 1-2028.csv to TP 1-2028 (1).csv


In [9]:
# CSV 파일 읽기
df = pd.read_csv('TP 1-2028.csv', encoding='utf-8')  # 인코딩이 다를 경우 적절히 변경하세요

In [10]:
# 'text' 열에서 데이터 추출 및 전처리
preprocessed_documents = df['text'].dropna().tolist()

In [11]:
# 커스텀 토크나이저 정의
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.morphs(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [12]:
# Mecab 토크나이저 초기화 및 벡터화
custom_tokenizer = CustomTokenizer(Mecab())
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, max_features=3000)
train_bow_embeddings = vectorizer.fit_transform(preprocessed_documents)

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
# 어휘 사전 생성
vocab = vectorizer.get_feature_names()
id2token = {k: v for k, v in zip(range(0, len(vocab)), vocab)}

In [ ]:
# BERT 임베딩 생성
train_contextualized_embeddings = bert_embeddings_from_list(preprocessed_documents,
                                                            "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens")

In [ ]:
# 토픽 모델 데이터 준비
qt = TopicModelDataPreparation()
training_dataset = qt.load(train_contextualized_embeddings, train_bow_embeddings, id2token)

In [ ]:
# 모델 훈련
ctm = CombinedTM(bow_size=len(vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

In [ ]:
# 결과 출력
print("Top 5 topics:")
print(ctm.get_topics(5))
print("\nTop 10 topics:")
print(ctm.get_topics(10))

In [ ]:
# LDAvis 시각화
lda_vis_data = ctm.get_ldavis_data_format(vocab, training_dataset, n_samples=10)
ctm_pd = vis.prepare(**lda_vis_data)
vis.display(ctm_pd)

In [ ]:
# 결과 저장
vis.save_html(ctm_pd, 'topic_model_visualization.html')
print("Visualization saved as 'topic_model_visualization.html'")

In [ ]:
# 토픽 할당 결과를 원본 데이터프레임에 추가
topics, _ = ctm.get_topic_lists(training_dataset)
df['assigned_topic'] = topics

In [ ]:
# 결과를 CSV 파일로 저장
df.to_csv('topic_modeling_results.csv', index=False)
print("Results saved to 'topic_modeling_results.csv'")